In [ ]:
from sympy import *
from sympy.solvers.solveset import linsolve

# Изразяваме неизвестните спрямо другите неизвестни от системата

def resolve_system_equations(array_of_equations, array_of_variables):
    results = []
    for index, system in enumerate(array_of_equations):
        array_of_variables[index] = resolve_equation(system, array_of_variables[index])
        for key, value in array_of_variables[index].items():
            new_result = {}
            new_result[key] = value
            results.append(new_result)
            try:
                array_of_equations[index+1] = array_of_equations[index+1].replace(str(key), str(value))
            except IndexError:
                return results[::-1]
    return results[::-1]

resolve_system_equations(['x - 2*y + 3*v - 7', '2*x + y + v- 4', '-3*x + 2*y -2*v + 10'], ['x', 'y', 'v'])

from sympy import *
from sympy.solvers.solveset import linsolve
import re
import string

# Заместваме неизвестните във всяко уравнение, чрез неизвестните от другите уравнения

def substitute_variables_in_system(system_results):
    for index, system in enumerate(system_results):
            for key, value in system_results[index].items():
                while re.search('[a-' + string.ascii_lowercase[string.ascii_lowercase.index(key)-1] +']['+ string.ascii_lowercase[string.ascii_lowercase.index(key)+1] +'-z]', value):
                    print('111')
                    for index1, system1 in enumerate(system_results):
                        if index1 != index:
                            for key1, value1 in system_results[index1].items():
                                value = value.replace(key1, value1)
                                system_results[index][key] = value
    return system_results

substitute_variables_in_system([{'v': '9*x/4 - 27/4'}, {'y': '5*v/3 - 1'}, {'x': '-3*v + 2*y + 7'}])

from sympy import *
from sympy.solvers.solveset import linsolve

# Пресмятаме всички уравнения

def calculate_final_variables_in_system(variables_to_be_calculated):
    final_results = []
    for index, system in enumerate(variables_to_be_calculated):
        for key, value in variables_to_be_calculated[index].items():
            final_results.append(resolve_equation(value + ' + ' + key, key))
    return final_results

calculate_final_variables_in_system([{'z': '-5*x/2 + 47/6'},
 {'y': '2*(-5*x/2 + 47/6)/5 - 3/5'},
 {'x': '2*(2*(-5*x/2 + 47/6)/5 - 3/5) - 3*(-5*x/2 + 47/6) + 7'}])

In [1]:
from sympy import *
from sympy.solvers.solveset import linsolve
from sympy import Symbol

# Уравнение

def resolve_equation(text, unknown_variable):
    x = Symbol(unknown_variable)
    equation = sympify(text)
    dict_equation = solve(equation, x)
    for res in dict_equation:
        result = {}
        result[unknown_variable] = str(res)
    return result

resolve_equation('2*(((1)/(2)))+((18)/(36))+2*y**2', 'y')

{'y': 'sqrt(3)*I/2'}

In [2]:
from sympy import *
from sympy.solvers.solveset import linsolve
from sympy import Symbol

# Система

def resolve_system(systems_list, symbols_list):
    expresion_symbols = []
    expresion_systems = []
    
    for sym in symbols_list:
        expresion_symbols.append(Symbol(sym))
        
    for sys in systems_list:
        expresion_systems.append(sympify(sys))
        
    system_results = solve(expresion_systems, expresion_symbols)
        
    return system_results

resolve_system(['2*(((1)/(2)))+((18)/(36))+2*y**2'], ['y'])

[(-sqrt(3)*I/2,), (sqrt(3)*I/2,)]

In [7]:
import re

# Преправяме текста, за да се реши от Sympy

def transform_text_to_sympy_text(equation_text):
    equation_text = equation_text.replace(" ", "")
    
    # Заместваме всички степени
    equation_text = replace_power(equation_text)
    
    # Заместваме всички дроби
    fraction_list_from_text = re.findall('frac{.*?}{.*?}', equation_text)
    for index, fraction in enumerate(fraction_list_from_text):
        equation_text = equation_text.replace(fraction_list_from_text[index], replace_fractions(fraction)) 
    
    # Добавяме умножение
    equation_text = add_multiply_for_sympy(equation_text)
    
    return equation_text
    
transform_text_to_sympy_text('2(frac {1} {2}) + frac {18} {36} + 2y^2')

'2*(((1)/(2)))+((18)/(36))+2*y**2'

In [6]:
import re

# Заместваме дроби, за да се решат от Sympy

def replace_fractions(equation_text):
    numerator = equation_text[equation_text.find("frac{")+5:equation_text.find("}")]
    equation_text = re.sub(r'(frac{([a-z]|[0-9])+}+)', '(('+numerator+')', equation_text)
    
    denominator = equation_text[equation_text.find(numerator+"{")+(len(numerator) + 5):equation_text.find("}")]
    equation_text = re.sub(r'({([a-z]|[0-9])+}+)', '/' + '(' + denominator + '))', equation_text)
    
    return equation_text

replace_fractions('frac{masdx}{3123}')

'((masdx)/(3123))'

In [5]:
import re

# Заместваме степенуване, за да се решат от Sympy

def replace_power(equation_text):
    
    equation_text = equation_text.replace('^', '**') 
    
    return equation_text

replace_power('(frac{masdx}{3123})^2')

'(frac{masdx}{3123})**2'

In [4]:
import re

# Добавяме умножение, за да се решат от Sympy

def add_multiply_for_sympy(equation_text):
    
    chars_to_replace = []
    
    for index, char in enumerate(equation_text):
        try:
            if (char.isalpha() and equation_text[index+1].isalpha()) or (char.isalpha() and equation_text[index+1].isdigit()) or (char.isdigit() and equation_text[index+1].isalpha()) or (char.isalpha() and equation_text[index+1] == '(') or (char == ')' and equation_text[index+1].isalpha()) or (char == ')' and equation_text[index+1].isdigit() or (char.isdigit() and equation_text[index+1] == '(')):
                new_char_to_replace = {}
                new_char_to_replace[str(char+equation_text[index+1])] = str(char + '*' + equation_text[index+1])
                chars_to_replace.append(new_char_to_replace)
        except IndexError:
            break
    
    for char_to_replace in chars_to_replace:
        for key, value in char_to_replace.items():
            equation_text = equation_text.replace(key, value)
        
    return equation_text

add_multiply_for_sympy('((1)/(2))+((18)/(36))+3(5 + 3)x')

'((1)/(2))+((18)/(36))+3*(5 + 3)*x'